# Optimize system with parameters

This is a template notebook for design of experiments for bayesian optimization.

Author: {{ cookiecutter.author_name }}
Created: {{ cookiecutter.timestamp }}

## How to use the notebook

The following cells:
- specify objective and other metrics, the parameter search space, and constraints,
- set up the optimization algorithm,
- read prior results,
- suggest new trials,
- provide the current best guess for optimal parameters.

Thereby, the library Ax is used, c.f. https://ax.dev/

By default, the notebook is set up to run with an example. To see how it works, run the notebook (multiple times) without changing the code.

For your project, adjust the code in the linked cells with your objectives, variables, dataset etc. and then execute all cells in order.

Please refer to bayesian_optimization.board for detailed instructions.

In [0]:
# Link to project experiments folder hypothesis_experiment_learnings.board (refresh and hit enter on this line to see the link)

### Imports and general setup

In [0]:
import os

from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from ax.service.ax_client import AxClient
from ax import RangeParameter, ChoiceParameter
from ax.exceptions.core import DataRequiredError, SearchSpaceExhausted
from ax.exceptions.generation_strategy import MaxParallelismReachedException
from ax.core.base_trial import TrialStatus
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models, ModelRegistryBase

import ax.plot as ax_plot

plt.style.use("dark_background")


## Project

In [0]:
experiment_name = '{{cookiecutter.use_case_name}}'  # please provide a name for the optimization experiment
data_dir = "./"           # please provide a name for saving the trial data for the experiment

data_file_name = os.path.join(data_dir,  f"data_{experiment_name}_running_trials.csv")
print(f"the trial data will be read from/stored in: {data_file_name}")

best_parameters_file_name = os.path.join(data_dir,  f"data_{experiment_name}_best_parameters.csv")
print(f"the best parameters will be read from/stored in: {best_parameters_file_name}")


## Metrics and objective

In [0]:
metrics = ["cost", "quality"]   # please provide a list of metrics
objective_name = "cost"         # please give the name for the objective to maximize or minimize (must be among provided metrics)
minimize = True                 # set to True if minimize, and to False if maximize objective

if objective_name not in metrics:
    raise ValueError(f"Objective must be among provided metrics. "
                     f"Could not find objective_name={objective_name} in metrics={metrics}")


## Parameters

In [0]:
parameters = [
    # please insert information on parameters, their names, types, bounds, etc.
    {"name": "x1",   
     "type": "range",
     "bounds": [-1.0, 1.0],
     "value_type": "float",  # Optional, defaults to inference from type of "bounds".
     "log_scale": False,  # Optional, defaults to False.
    },
    {"name": "x2",   
     "type": "range",
     "bounds": [-1.0, 1.0],
     "value_type": "float",  # Optional, defaults to inference from type of "bounds".
     "log_scale": False,  # Optional, defaults to False.
    },
    {"name": "x3",   
     "type": "range",
     "bounds": [-1.0, 1.0],
     "value_type": "float",  # Optional, defaults to inference from type of "bounds".
     "log_scale": False,  # Optional, defaults to False.
    },
]


## Constraints

In [0]:
parameter_constraints = ["x1 + x2 <= 10"]      # provide any parameter constraints as inequalities
outcome_constraints = ["quality >= 1"]         # provide any constraints on the metrics

## Schedule

In [0]:
suggest_new_trials = True         # set to `True` if you want new trials suggested, 
                                  # set to `False` if you just want to use existing results to estimate best parameters 


max_batch_size = 10               # please provide the max. number of trials in a batch
always_max_batch_size = True      # whether to force full batch size for suggested new trials
suggest_when_outstanding = False  # whether to suggest when trials are still outstanding

initial_n_trials = 5              # how many initial trials before Baysian optimization steps


## Trial generation and best parameter estimation

### Complete outstanding trials

Note that the following cell contains code to invent results of any outstanding trials for demonstration purposes. 

For real applications, either
 - replace the cell with appropriate code for retrieving the actual trial results, or 
 - remove the cell entirely, if you intend to add the trial results to the data files in a different way.

In [0]:
if os.path.exists(data_file_name):
    data = pd.read_csv(data_file_name, index_col="index")

    data["cost_mean"] = data["x1"]**2 + data["x2"]**2 + data["x3"]**2
    data["cost_SEM"] = 0.1
    data["quality_mean"] = 2./(1 + np.exp(-data["x3"] + 0.2))
    data["quality_SEM"] = 0.01

    display(data)

    data.to_csv(data_file_name)


### Read any existing data

In [0]:
parameter_columns = [parameter["name"] for parameter in parameters ] 
result_columns    = [metric + suffix for metric in metrics for suffix in ("_mean", "_SEM")]
data_columns      = parameter_columns + result_columns

n_trials = 0
n_completed_trials = 0
n_outstanding_trials = 0
prior_data = None

if os.path.exists(data_file_name):
    print(f"reading prior data from {data_file_name}...")
    prior_data = pd.read_csv(data_file_name, index_col="index")

    missing_colums = set(data_columns) - set(prior_data.columns)
    if missing_colums:
        raise ValueError(f"data file missing colums: {missing_colums}.")
    prior_data = prior_data[data_columns]   

    n_trials = len(prior_data[parameter_columns].dropna(axis='index', how='any'))
    n_completed_trials = len(prior_data.dropna(axis='index', how='any'))
    n_outstanding_trials = n_trials - n_completed_trials

else:
    print("no prior data.")

### Set up client

In [0]:


generation_strategy_steps=[
        # 1. Initialization step (does not require pre-existing data and is well-suited for 
        # initial sampling of the search space)
        GenerationStep(
            model=Models.SOBOL,
            num_trials=max(max_batch_size, initial_n_trials) if always_max_batch_size else initial_n_trials,  # How many trials should be produced from this generation step
            min_trials_observed=3, # How many trials need to be completed to move to next model
            max_parallelism=max(max_batch_size, 5) if always_max_batch_size else 5,  # Max parallelism for this step
            model_kwargs={"seed": 999},  # Any kwargs you want passed into the model
            model_gen_kwargs={},  # Any kwargs you want passed to `modelbridge.gen`
        ),
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=Models.GPEI,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=max(3, max_batch_size) if always_max_batch_size else 3,  # Parallelism limit for this step, often lower than for Sobol
        ),
    ]


if n_trials >= initial_n_trials:
    generation_strategy = GenerationStrategy(generation_strategy_steps[1:])
else:
    generation_strategy = GenerationStrategy(generation_strategy_steps)


ax_client = AxClient(
    generation_strategy=generation_strategy,
    enforce_sequential_optimization=not always_max_batch_size, )

ax_client.create_experiment(
    name=experiment_name,
    parameters=parameters,
    objective_name=objective_name,
    minimize=minimize,
    parameter_constraints=parameter_constraints,
    outcome_constraints=outcome_constraints,
)


### Feed existing data to client

In [0]:
prior_trials = dict()
if prior_data is not None:
    for index, trial_data in prior_data.iterrows():

        trial_parameters = trial_data[parameter_columns]
        if any(trial_parameters.isna()):
            missing_trial_parameters = ", ".join(trial_parameters[trial_parameters.isna()].index)
            print(f"row {index}: missing parameter values for: {missing_trial_parameters}.")
            continue

        trial_parameters = trial_parameters.to_dict()
        trial_parameters, trial_index = ax_client.attach_trial(parameters=trial_parameters)

        trial_results = trial_data[result_columns]
        if any(trial_results.isna()):
            missing_results = ", ".join(trial_results[trial_results.isna()].index)
            print(f"row {index}: outstanding results for: {missing_results}.")
        else:
            raw_data = dict()
            for metric in metrics:
                metric_mean = trial_results[metric + "_mean"]
                metric_SEM  = trial_results[metric + "_SEM"]
                raw_data[metric] = (metric_mean, metric_SEM)
            ax_client.complete_trial(trial_index=trial_index, raw_data=raw_data)

        trial_results = trial_results.to_dict()
        prior_trials[trial_index] = {**trial_parameters, **trial_results}


### Suggest new trials

In [0]:
if (n_outstanding_trials > 0) and not suggest_when_outstanding:
    print(f"There are {n_outstanding_trials} outstanding trials. Will not suggest new trials.")
    suggest_new_trials = False


if suggest_new_trials and max_batch_size > 0:

    new_trials = dict()
    exhausted = False

    try:
        if always_max_batch_size:
            for _ in range(max_batch_size):
                trial_parameters, trial_index = ax_client.get_next_trial()
                trial_results = {c: None for c in result_columns}
                trial_data = {**trial_parameters, **trial_results}
                new_trials[trial_index] = trial_data
        else:
            # workaround (get_next_trials won't generate new trials sometimes if loaded from disk unless get_next_trial was called)
            trial_parameters, trial_index = ax_client.get_next_trial()
            trial_results = {c: None for c in result_columns}
            trial_data = {**trial_parameters, **trial_results}
            new_trials[trial_index] = trial_data
            if max_batch_size > 1:
                more_trials, exhausted = ax_client.get_next_trials(max_trials=max_batch_size - 1)
                new_trials.update(more_trials)
    except (DataRequiredError, SearchSpaceExhausted, MaxParallelismReachedException) as exception:
        print(f"no more trials because {type(exception).__name__}: {exception}")
        pass
        
    _, exhausted = ax_client.get_current_trial_generation_limit()

    batch_size = len(new_trials)

    if (batch_size <= 0) and exhausted:
        print("exhausted the search. no more trials to suggest.")

    elif batch_size <= 0:
        print("no new trials to suggest. maybe you have to complete outstanding trials first.")

    else:
        print(f"got {batch_size} new trials.")

        if os.path.exists(data_file_name):
            dt = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
            os.rename(data_file_name, os.path.join(data_dir,  f"data_{experiment_name}_running_trials_{dt}.csv"))

        data = {**prior_trials, **new_trials}
        data = pd.DataFrame.from_dict(data, orient='index')
        data = data[data_columns]
        data.index.name = "index"
        data.to_csv(data_file_name)



### Estimate best parameters

In [0]:
if os.path.exists(best_parameters_file_name):
    prior_best_parameters_data = pd.read_csv(best_parameters_file_name) 
else:
    prior_best_parameters_data = pd.DataFrame(columns=["n_completed_trials"] + parameter_columns + metrics)


best_parameters_result = ax_client.get_best_parameters()
if best_parameters_result is None:
    best_parameters = None
    means = None
    covariances = None
    new_best_parameters_data = pd.DataFrame(columns=["n_completed_trials"] + parameter_columns + metrics)
else:
    best_parameters, (means, covariances) = best_parameters_result
    new_best_parameters_data = pd.DataFrame.from_records(({
        "n_completed_trials": n_completed_trials,
        **best_parameters, **means
    },))


best_parameters_data = prior_best_parameters_data.append(new_best_parameters_data)
if os.path.exists(best_parameters_file_name):
    dt = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    os.rename(best_parameters_file_name, os.path.join(data_dir,  f"data_{experiment_name}_best_parameters_{dt}.csv"))
best_parameters_data.to_csv(best_parameters_file_name, index=False)

if len(best_parameters_data) > 0:
    print("\nbest parameters so far (from oldest to most recent):")
    display(best_parameters_data)
else:
    print("no best parameters yet.")



### Plot results

In [0]:
experiment = ax_client.experiment

ob_trials =  {i: t.objective_mean for i,t in experiment.trials.items() if t.completed_successfully}
ob_trials = [ob_trials[i] for i in sorted(ob_trials.keys())]

if minimize:
    best_ob_trials = np.minimum.accumulate(ob_trials)
else:
    best_ob_trials = np.maximum.accumulate(ob_trials)

fig, axs = plt.subplots(1, 2, figsize=(6 * 2, 4))


ax = axs[0]
ax.plot(ob_trials, '.r');
ax.set_xlabel('trial number')
ax.set_ylabel(objective_name)


ax = axs[1]
ax.plot(best_parameters_data["n_completed_trials"], best_parameters_data[objective_name], '.r');
ax.set_xlabel('number of completed trials');
ax.set_ylabel('best estimated ' + objective_name);
xmax = best_parameters_data["n_completed_trials"].max() if (len(best_parameters_data["n_completed_trials"]) > 0) else 0.
ax.set_xlim(-0.5, xmax + 0.5);
